第三节：解析库的使用——lxml bs4 and PyQuery.（静态爬取模块）
==

对于网页的节点来说，它可以定义id、class 或其他属性。
而且节点之间还有层次关系，在网页中可以通过XPath或css选择器来定位一个或多个节点。
那么，在页面解析时，利用XPath或css选择器来提取某个节点，然后再调用相应方法获取它的正文内容或者属性，
不就可以提取我们想要的任意信息了吗？

# 1.XPath解析

这块内容和XML教程相通，参考资料：[https://www.w3school.com.cn/xpath/index.asp](https://www.w3school.com.cn/xpath/index.asp)

例子： **//title[@lang=’eng']**
这就是一个 XPath 规则 ，它代表选择所有名称为 title，同时属性 lang 的值为 eng 的节点。

**外部库：lxml解析库**

In [9]:
from lxml import etree
originalText ='''
<div>
<ul>
<li class='item-0'><a herf='link1.html'>first item
<li class='item-1'><a href='link2.html'>second Item</a>
<li class='item-2'><a href='link3.html'>third Item</a></li>
    <li class='item-3'><a href='link4.html'>fifth item</a></li>
</div>
'''
html = etree.HTML(originalText)
print(html)
print(etree.tostring(html).decode('utf-8'))

<Element html at 0x13b5b3617c0>
<html><body><div>
<ul>
<li class="item-0"><a herf="link1.html">first item
<li class="item-1"><a href="link2.html">second Item</a>
</li><li class="item-2"><a href="link3.html">third Item</a></li>
    <li class="item-3"><a href="link4.html">fifth item</a></li>
</a></li></ul></div>
</body></html>


这里首先导入lxml库的etree模块，然后声明了一段HTML文本，
调用HTML类进行初始化，这样就成功构造了一个 XPath 解析对象。
这里需要注意的是，HTML文本中的有些节点是没有闭合的，
但是etree模块可以自动修正HTML文本。

这里我们调用 tostring方法即可输出修正后的 HTML代码，
但是结果是bytes类型。 这里利用decode方法将其转成str类

上面的代码实现了对不完整的html格式文本的自动补全技术

我们再查看一下lxml相关参数

In [51]:
pip show lxml

Name: lxml
Version: 4.5.1
Summary: Powerful and Pythonic XML processing library combining libxml2/libxslt with the ElementTree API.
Home-page: https://lxml.de/
Author: lxml dev team
Author-email: lxml-dev@lxml.de
License: BSD
Location: f:\anaconda\envs\python38\lib\site-packages
Requires: 
Required-by: pyquery
Note: you may need to restart the kernel to use updated packages.


In [52]:
with open('Lxmlparse.html','w') as file:
    file.write(originalText)
file.close()
html = etree.parse('Lxmlparse.html',etree.HTMLParser())
res = etree.tostring(html)
print(res.decode('utf-8'))

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN" "http://www.w3.org/TR/REC-html40/loose.dtd">
<html><body><div>&#13;
<ul>&#13;
<li class="item-0"><a herf="link1.html">first item&#13;
<li class="item-1"><a href="link2.html">second Item</a>&#13;
</li><li class="item-2"><a href="link3.html">third Item</a></li>&#13;
    <li class="item-3"><a href="link4.html">fifth item</a></li>&#13;
</a></li></ul></div>&#13;
</body></html>


这次的输出结果略有不同，多了一个DOCTYPE的声明，不过对解析无任何影响,表明遵循XML的DTD协议，将文本更加标准化输出

同时对文本的换行符\n转化为&#13;了。同样这也是XML相关的语法。

我们一般会用\\开头的XPath 规则来选取所有符合要求的节点。 这里以前面的HTML文本为例，如果要选取所有节点，可以这样实现：

In [53]:
res = html.xpath('//*')
res

[<Element html at 0x1f05099d580>,
 <Element body at 0x1f0509a0400>,
 <Element div at 0x1f0509a0380>,
 <Element ul at 0x1f0509a0580>,
 <Element li at 0x1f0509a0500>,
 <Element a at 0x1f0509a05c0>,
 <Element li at 0x1f0509a0600>,
 <Element a at 0x1f0509a0640>,
 <Element li at 0x1f0509a0680>,
 <Element a at 0x1f0509a0540>,
 <Element li at 0x1f0509a06c0>,
 <Element a at 0x1f0509a0700>]

这里使用＊代表匹配所有节点，也就是整个 HTML 文本中的所有节点都会被获取。可以看到，返回形式是一个列表，每个元素是 Element 类型，其后跟了节点的名称

同理，如果我们想要获取所有的li或者a下的所有节点，可以使用xpath转化成想要的元素

In [54]:
res = html.xpath('//li[@class]')
res

[<Element li at 0x1f0509a0500>,
 <Element li at 0x1f0509a0600>,
 <Element li at 0x1f0509a0680>,
 <Element li at 0x1f0509a06c0>]

上面的返回都是表示元素的对象，下面我们需要获取对应的节点的文本

### 任务： 获取属性class='item-2'的文本？

In [55]:
res = html.xpath(".//li[@class='item-2']/text()")
res

[]

没有获得文本？

其中一个节点因为自动修正，li节点的尾标签添加的时候换行了，所以提取文本得到的唯一结果，就是li节点的尾标签和a节点的尾标签之间的换行符。
或者直接没有数据。下面做一些修正

In [57]:
#方法1，逐层获取,利用层次结构实现
res = html.xpath("//li[@class='item-2']/a/text()")
res

['third Item']

In [56]:
#方法2：
res = html.xpath("//li[@class='item-2']//text()")
res

['third Item']

两种方法建议使用方法一，因为方法2//选取所有的子孙文本，导致多了不要的字符数据，影响结果，
比如换行符等等

下面是属性获取：

In [10]:
res = html.xpath("//li/a/@href")
res

['link2.html', 'link3.html', 'link4.html']

上面的代码返回的是所有href属性的内容

下面的代码实现内容包含多个属性关键字的情景

In [11]:
text = '''
<li class='li li-first'><a href='a.html'>nnn</a></li>
<li class='li li-first' name="item"><a href='b.html'>mmm</a></li>
'''
html = etree.HTML(text)
res = html.xpath('//li[contains(@class,"li")]/a/text()')
res

['nnn', 'mmm']

遇到多属性的情景，我们使用and语句实现交的要求

In [12]:
res = html.xpath('//li[contains(@class,"li") and @name="item"]/a/text()')
res

['mmm']

## 按序选择
判定谓词使用方括号 [...] 进行表示：对指定关系轴的、满足节点测试的所有节点，使用判定谓词规定的条件进行筛选。如果判定谓词为一个整数 i（或是经过计算得到的），则表示选择序列中的第 i 个元素。例如：

**/library/book[1] 表示选择 library 中的第一个 book 元素，
    等价于 /library/book[position() = 1]；**

**/library/book[@name = “TCP/IP”] 表示选择属性 name 的值为
    “TCP/IP” 的 book 元素**

In [13]:
originalText ='''
<div>
<ul>
<li class='item-0'><a herf='link1.html'>first item</a>
<li class='item-1'><a href='link2.html'>second Item</a>
<li class='item-2'><a href='link3.html'>third Item</a></li>
    <li class='item-3'><a href='link4.html'>fifth item</a></li></ul>
</div>
'''
from lxml import etree
html = etree.HTML(originalText)
res = html.xpath('//li[1]/a/text()')
res

['first item']

上面代码表示：
我们选取了第一个li节点，中括号中传入数字1即可。 注意，这里和代码中不同，序号是以1开头的，不是以0开头。

## 节点轴选择

节点测试可以是名称测试或者类型测试。

名称测试表示根据指定的名称对当前节点进行测试；可以使用带命名空间限制的完整的元素名称。

类型测试则允许根据节点的类型、以及在 Schema 中定义的数据类型进行测试（仅适用于元素和属性节点）；有些类型的节点（如注释节点和文本节点）是没有名称的，无法使用名称测试，所以只能使用类型测试的方式。

名称测试：

/library/descendant::book 表示查找 /library 元素的所有名为 book 的子孙节点

类型测试：

/library/child::node() 表示选择指定关系轴下的所有节点

/library/book/text() 表示查找 book 元素下的所有文本节点

通配符：

/library/book/*  表示book元素下的所有子元素节点

In [14]:
from lxml import etree
html = etree.HTML(originalText)
res = html.xpath('//li[1]/a/ancestor::div')
res

[<Element div at 0x13b5aeb7840>]

In [15]:
#sibling 同级节点
res = html.xpath('//li[1]/following-sibling::*')
res

[<Element li at 0x13b5acd12c0>,
 <Element li at 0x13b5b167c80>,
 <Element li at 0x13b5b167300>]

至此，lxml相关大致结束。其本质就是利用XML的XPath语言进行筛选从而方便地查找，查询相关用法，可以进入访问url:
[http://lxml.de/](http://lxml.de/)

# 2.BeautifulSoup解析

BeautifulSoup,即“美味汤”，借助网页的结构和属性等特性来解析网页。有了它，我们不用再去写一些复杂的正则表达式，只需要简单的几条语句，就可以完成网页中某个元素的提取。

In [16]:
pip show beautifulsoup4

SyntaxError: invalid syntax (<ipython-input-16-a7a6b9e0f7db>, line 1)

简单来说，BeautifulSoup就是Python的一个HTML或XML的解析库，可以用它来方便地从网页中提取数据。

相关官方文档：[https://beautifulsoup.readthedocs.io/zh_CN/v4.4.0/](https://beautifulsoup.readthedocs.io/zh_CN/v4.4.0/)

**官方解释如下：**

BeautifulSoup提供一些简单的、
Python式的函数来处理导航、搜索、修改分析树等功能。

它是一个工具箱，通过解析文档为用户提供需要获取的数据，
因为简单，所以不需要多少代码就可以写出一个完整的应用程序。

BeautifulSoup自动将输入文档转换为Unicode编码，输出文档转换为 UTF-8 编码。
不需考虑、编码方式，除非文档没有指定一个编码方式，这时你仅仅需说明一下原始编码方式就可以了 。

BeautifulSoup 已成为和lxml、html6lib 一样出色的Python解释器，为用户灵活地提供不同的解析策略或强劲的速度。

BeautifulSoup支持的解析器：

<font face="黑体" color=red size=4>
  1. 标准库：html.pareser  执行速度适中，但是Python早期版本3.2.2前容错差

2. lxml解析器（包括lxml与xml）唯一支持xml的解析器，都要装C语言库

3. html5lib 容错好、以浏览器界面显示 生成h5格式文档，但是速度慢，不依赖外部扩展
</font>

In [5]:
#样例html文档
html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story"><a href='www.baidu.com'>...</a></p>
"""
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc,'lxml')
soup.p.string

"The Dormouse's story"

## 规整化打印：

In [3]:
#规格化输出
print(soup.prettify())

<html>
 <head>
  <title>
   The Dormouse's story
  </title>
 </head>
 <body>
  <p class="title">
   <b>
    The Dormouse's story
   </b>
  </p>
  <p class="story">
   Once upon a time there were three little sisters; and their names were
   <a class="sister" href="http://example.com/elsie" id="link1">
    Elsie
   </a>
   ,
   <a class="sister" href="http://example.com/lacie" id="link2">
    Lacie
   </a>
   and
   <a class="sister" href="http://example.com/tillie" id="link3">
    Tillie
   </a>
   ;
and they lived at the bottom of a well.
  </p>
  <p class="story">
   ...
  </p>
 </body>
</html>


In [21]:
#输出文本a标签下的第一个节点内容
soup.a.string

'Elsie'

In [22]:
type(soup.title)

bs4.element.Tag

In [23]:
#title 节点匹配
soup.title

<title>The Dormouse's story</title>

In [24]:
soup.a.attrs['href']

'http://example.com/elsie'

In [25]:
#获取属性值
soup.a['href']

'http://example.com/elsie'

In [26]:
#获取所有属性值，返回一个字典类型数据
soup.a.attrs

{'href': 'http://example.com/elsie', 'class': ['sister'], 'id': 'link1'}

In [34]:
type(soup)


bs4.BeautifulSoup

这里需要注意的是，有的返回结果是字符串，有的返回结果是字符串组成的列表。
比如，name属性的值是唯一的，返回的结果就是单个字符串。
而对于class，一个节点元素可能有多个class，所以返回的是列表。
在实际处理过程中，我们要注意判断类型。

In [5]:
#嵌套选择
soup.html.head

<head><title>The Dormouse's story</title></head>

# 关联选择技术

在做选择的时候，有时候不能做到一步就选到想要的节点元素，
需要先选中某一个节点元素，然后以它为基准再选择它的子节点、父节点、兄弟节点等，
这里就来介绍如何选择这些节点元素。


In [8]:
soup.head.contents


[<title>The Dormouse's story</title>]

返回结果是列表形式。如果节点里既包含文本，又包含节点，最后会将它们以列表形式统一返回。

需要注意的是，列表中的每个元素都是p节点的直接子节点。 比如第一个 a 节点里面包含一层 span 节点，这相当于孙子节点了，但是返回结果并没有单独把span节点选出来。

contents属性得到的结果是直接子节点的列表。


In [18]:
#子节点遍历技术
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc,'lxml')
print(soup.p.children)
for i,child in enumerate(soup.p.children):
    print(i, child)

0 <b>The Dormouse's story</b>


获取父节点：使用parent属性；如果想获取所有的祖先节点，可以调用parents属性：

调用了 children 属性来选择子节点；
descendants会递归查询所有子节点，得到所有的子孙节点。

next_sibling 和 previous_sibling 分别获取节点的下一个 和上一个兄弟元素， next_siblings 和 previous_siblings 则分别返回所有前面和后面的兄弟节点的生成器。

下面实现：所有属性的获取，包括文本属性等

In [20]:
print(soup.a.next_siblings)
for i in list(soup.a.next_siblings):
    print(i)
print(list(soup.a.parents))

<generator object PageElement.next_siblings at 0x0000013B5BB85C80>
,

<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>
 and

<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>
;
and they lived at the bottom of a well.
[<p class="story">Once upon a time there were three little sisters; and their names were
<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>, <body>
<p class="title"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
and they lived at 

前面所讲的选择方法都是通过属性来选择的，这种方法非常快，
但是如果进行比较复杂的选择的话，它就比较烦琐，不够灵活了。
幸好，BeautifulSoup还为我们提供了一些查询方法，比如 **find_all()**
和 **find()** 等，调用它们，然后传入相应的参数，就可以灵活查询了。

find_all，顾名思义，就是查询所有符合条件的元素。给它传入一些属性或文本，
就可以得到符合条件的元素，它的功能十分强大。

## API: find_all(name,attrs,recursive,text,**kwargs)

In [6]:
html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story"><a href='www.baidu.com'>...</a></p>
"""
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc,'lxml')
print('携带a标签的html文本：') #注意：返回值是一个列表
print(soup.find_all('a'))
print('携带b的文本')
print(soup.find_all(name='b'))
print("进行遍历查询,查询p节点下的所有a节点得到的结果：")
for a in soup.find_all('p'):
    print(a.find_all(name='a'))
    for b in soup.find_all('a'):
        print(b.string)#获取文本：
print("进行属性查询......")
print("1.查找属性class=sister 且 id=link3 的文本列表")
print(soup.find_all(attrs={'class':'sister','id':'link3'}))
print('2.查找属性class=sister的文本列表')
print(soup.find_all(attrs={'class':'sister'}))
#注意属性的传参是字典类型的！
#相同方法，我们用直接属性比较实现：
print(soup.find_all(class_='sister'))
#注意：这里的class和Python关键字重名了，因此需要变成class_模拟变化。

携带a标签的html文本：
[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>, <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>, <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>, <a href="www.baidu.com">...</a>]
携带b的文本
[<b>The Dormouse's story</b>]
进行遍历查询,查询p节点下的所有a节点得到的结果：
[]
Elsie
Lacie
Tillie
...
[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>, <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>, <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]
Elsie
Lacie
Tillie
...
[<a href="www.baidu.com">...</a>]
Elsie
Lacie
Tillie
...
进行属性查询......
1.查找属性class=sister 且 id=link3 的文本列表
[<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]
2.查找属性class=sister的文本列表
[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>, <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>, <a class="sister" href="http://example.com/tillie" id

## 结合正则表达式查询相关文本数据，需要用到re.compile()实现面向对象操作！


In [9]:
import re
print("提取文本中：总长度为5且后两位是ie的文本：")
print(soup.find_all(text=re.compile('^\w{3}ie$')))

提取文本中：总长度为5且后两位是ie的文本：
['Elsie', 'Lacie']


除了 find_all()方法，还有 find()方法，只不过后者返回的是单个元素，也就是第一个匹配的元素，而前者返回的是所有匹配的元素组成的列表

另外，还有许多查询方法，其用法与前面介绍的find_all()、find()方法完全相同，
只不过查询范围不同，这里简单说明一下。

**find_parents()**和**find_parent()**：前者返回所有祖先节点,后者返回直接父节点。

**find_next_siblings()**和**find_next_ sibling()**：前者返回后面所有的兄弟节点，
后者返回后面第一个兄弟节点。

**find_previous_siblings()**和**find_previous_sibling()**：
前者返回前面所有的兄弟节点，后者返回前面第一个兄弟节点。

## CSS选择器

BeautifulSoup还提供了另外一种选择器，那就是css选择器,相关文档参考：
[https://www.w3school.com.cn/cssref/css_selectors.asp](https://www.w3school.com.cn/cssref/css_selectors.asp)

使用css选择器时，只需要调用select()方法，传入相应的CSS选择器即可，

In [12]:
tml_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story"><a href='www.baidu.com'>...</a></p>
"""
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc,'lxml')
print(soup.select('p b'))
print(soup.select('.story .sister'))#匹配class=?
print(soup.select('#link1 .sister'))#匹配id号的
#获取文本
print("获取文本。。。。。。")
for i in soup.select('p'):
    print("Text: ",i.get_text())
    print("String: ",i.string)

[<b>The Dormouse's story</b>]
[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>, <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>, <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]
[]
获取文本。。。。。。
Text:  The Dormouse's story
String:  The Dormouse's story
Text:  Once upon a time there were three little sisters; and their names were
Elsie,
Lacie and
Tillie;
and they lived at the bottom of a well.
String:  None
Text:  ...
String:  ...


# PyQuery 探索更强大的选择器

如果你对 Web 有所涉及，如果你比较喜欢用CSS选择器，如果你对jQuery有所了解，
那么这里有一个更适合你的解析库——pyquery。

## 初始化pyquery

初始化 pyquery的时候，也需要传入HTML文本来初始化一个 PyQuery 对象。

它的初始化方式有多种，比如直接传入字符串，传入URL，传入文件名，等等。

In [11]:
html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story"><a href='www.baidu.com'>...</a></p>
"""
from pyquery import PyQuery
doc = PyQuery(html_doc)
print('打印a 标签数据:')
print(doc('a'))
print("a标签：")
doc('a')

打印a 标签数据:
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.<a href="www.baidu.com">...</a>
a标签：


[<a#link1.sister>, <a#link2.sister>, <a#link3.sister>, <a>]

In [7]:
#url初始化
from pyquery import PyQuery
#发送请求
doc = PyQuery(url='https://www.runoob.com/cssref/css-selectors.html')
print("Title 标签：")
print(doc('title'))
print('p标签：')
print(doc('p'))
#打印url下的title标签的数据

Title 标签：
<title>CSS 选择器 | 菜鸟教程</title>

  
p标签：
<p class="elem_d11">CSS选择器用于选择你想要的元素的样式的模式。</p>&#13;
<p class="elem_d11">"CSS"列表示在CSS版本的属性定义（CSS1，CSS2，或对CSS3）。</p>&#13;
<p><a id="cancel-comment-reply-link" href="javascript:;">取消</a></p>&#13;
			<p>感谢您的支持，我会继续努力的!</p>
      <p>打开<span id="shang_pay_txt">支付宝</span>扫一扫，即可进行扫码打赏哦</p>
        <p><a href="//c.runoob.com/codedemo/5348" target="_blank"><span style=" font-size: 14px;color: #000;font-weight: bold;">点我查看本站打赏源码！</span></a></p>
      <p class="fieldset">
					<label class="image-replace cd-username" for="signin-username">用户名</label>
					<input class="full-width has-padding has-border" id="signin-username" name="username" type="text" placeholder="输入用户名"/>
				</p>

				<p class="fieldset">
					<label class="image-replace cd-password" for="signin-password">密码</label>
					<input class="full-width has-padding has-border" id="signin-password" name="password" type="password" placeholder="输入密码"/>
				</p>
				
				<p class="fieldset">

PyQuery对象会首先请求这个URL，
然后用得到的HTML内容完成初始化，
这其实就相当于用网页的源代码以字符串的形式传递给PyQuery类来初始化。

等价于：

**doc =PyQuery(requests.get(url).text)**
**print(doc('title'))**

还可以传递本地的文件名，此时将参数指定为filename即可

In [9]:
#file gaining
doc =PyQuery(filename='Lxmlparse.html')
print(doc('li'))

<li class="item-0"><a herf="link1.html">first item
<li class="item-1"><a href="link2.html">second Item</a>
</li><li class="item-2"><a href="link3.html">third Item</a></li>
    <li class="item-3"><a href="link4.html">fifth item</a></li>
</a></li><li class="item-1"><a href="link2.html">second Item</a>
</li><li class="item-2"><a href="link3.html">third Item</a></li>
    <li class="item-3"><a href="link4.html">fifth item</a></li>



当然最常用的初始化方式还是以字符串形式传递。

## 使用CSS选择器+Pyquery实现编程

In [10]:
doc = PyQuery(html_doc)
print(doc('.story'))
print("Type:")
print(type(doc('#container .story p')))

NameError: name 'html_doc' is not defined

可以看到，CSS选择器下获取的类型依然是 PyQuery 类型。

下面简述子节点父节点和兄弟节点

In [3]:
from bs4 import BeautifulSoup
import requests
url = 'https://www.runoob.com/cssref/css-selectors.html'
r = requests.get(url)
soup = BeautifulSoup(r.text,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="465267610762567726375" property="qc:admins"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <title>
   CSS 选择器 | 菜鸟教程
  </title>
  <link href="//s.w.org" rel="dns-prefetch"/>
  <link href="http://www.runoob.com/cssref/css-selectors.html" rel="canonical"/>
  <meta content="CSS 选择器" name="keywords"/>
  <meta content="CSS 选择器  CSS选择器用于选择你想要的元素的样式的模式。 “CSS”列表示在CSS版本的属性定义（CSS1，CSS2，或对CSS3）。         选择器     示例     示例说明     CSS    	     .class     .intro     选择所有class=“intro”的元素     1    	     #id     #firstname     选择所有id='firstname.." name="description"/>
  <link href="https://static.runoob.com/images/favicon.ico" mce_href="//static.runoob.com/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="/wp-content/themes/runoob/style.css?v=1.156" media="all" rel="stylesheet" type="text/css"/>
  <link href="https://static.runoob.

In [12]:
doc = PyQuery(url=url)
item = doc('.mobile-nav')
print("类型：\n",type(item))
print('查找子孙节点文本：')
print(item)
lis = item.find('li')#查找find的文本,范围：查找所有子孙节点
print(lis)
print("子节点")
lis2 = item.children()#子节点
print(lis2)
print("父类节点：")
lis3 = item.parent('.row')
print(lis3)
print("祖先节点：")
lis4 = item.parents('.row')
print(lis4)

print("子节点的兄弟节点：")
lis5 =lis2.siblings()
print(lis5)

类型：
 <class 'pyquery.pyquery.PyQuery'>
查找子孙节点文本：
<ul class="mobile-nav">
				<li><a href="//www.runoob.com/">首页</a></li>
				<li><a href="/html/html-tutorial.html">HTML</a></li>
				<li><a href="/css/css-tutorial.html">CSS</a></li>
				<li><a href="/js/js-tutorial.html">JS</a></li>
				<li><a href="/browser-history">本地书签</a></li>
				<a href="javascript:void(0)" class="search-reveal">Search</a> 
			</ul>
			
		
<li><a href="//www.runoob.com/">首页</a></li>
				<li><a href="/html/html-tutorial.html">HTML</a></li>
				<li><a href="/css/css-tutorial.html">CSS</a></li>
				<li><a href="/js/js-tutorial.html">JS</a></li>
				<li><a href="/browser-history">本地书签</a></li>
				
子节点
<li><a href="//www.runoob.com/">首页</a></li>
				<li><a href="/html/html-tutorial.html">HTML</a></li>
				<li><a href="/css/css-tutorial.html">CSS</a></li>
				<li><a href="/js/js-tutorial.html">JS</a></li>
				<li><a href="/browser-history">本地书签</a></li>
				<a href="javascript:void(0)" class="search-reveal">Search</a> 

在使用的同时可以采用for循环进行遍历

## 信息获取上，我们可以获取属性或者文本

下面我们以上述子节点lis2为例：

In [15]:
print(lis2)
a = lis2('a')
print(a.attr.href)
#默认是查找第一个项目的，遍历要使用items方法。

print("遍历：")
for li in a.items():
    print(li.attr.href)

<li><a href="//www.runoob.com/">首页</a></li>
				<li><a href="/html/html-tutorial.html">HTML</a></li>
				<li><a href="/css/css-tutorial.html">CSS</a></li>
				<li><a href="/js/js-tutorial.html">JS</a></li>
				<li><a href="/browser-history">本地书签</a></li>
				<a href="javascript:void(0)" class="search-reveal">Search</a> 
			
//www.runoob.com/
遍历：
//www.runoob.com/
/html/html-tutorial.html
/css/css-tutorial.html
/js/js-tutorial.html
/browser-history
javascript:void(0)


In [16]:
#文本同样使用text()方法实现：
print("所有文本：")
print(lis2.text())
print("遍历：")
for i in lis2.items():
    print("文本：",i.text())
    print("html文本：",i.html())

所有文本：
首页 HTML CSS JS 本地书签 Search
遍历：
文本： 首页
html文本： <a href="//www.runoob.com/">首页</a>
文本： HTML
html文本： <a href="/html/html-tutorial.html">HTML</a>
文本： CSS
html文本： <a href="/css/css-tutorial.html">CSS</a>
文本： JS
html文本： <a href="/js/js-tutorial.html">JS</a>
文本： 本地书签
html文本： <a href="/browser-history">本地书签</a>
文本： Search
html文本： Search


## 节点操作与动态修改

<font face="黑体" color=purple size=4>1. addClass & removeClass</font>

In [39]:
print("原始html")
print(lis2)
lis2.add_class('active')
lis2.add_class('item-0')
print("添加class属性后：")
print(lis2)
print("去除active 的 class属性后：")
lis2.remove_class('active')
print(lis2)

原始html
<li class="item-0" name="link">changedIttem</li>
				<li class="item-0" name="link">changedIttem</li>
				<li class="item-0" name="link">changedIttem</li>
				<li class="item-0" name="link">changedIttem</li>
				<li class="item-0" name="link">changedIttem</li>
				<a href="javascript:void(0)" class="item-0 search-reveal" name="link">changedIttem</a> 
			
添加class属性后：
<li class="item-0 active" name="link">changedIttem</li>
				<li class="item-0 active" name="link">changedIttem</li>
				<li class="item-0 active" name="link">changedIttem</li>
				<li class="item-0 active" name="link">changedIttem</li>
				<li class="item-0 active" name="link">changedIttem</li>
				<a href="javascript:void(0)" class="item-0 search-reveal active" name="link">changedIttem</a> 
			
去除active 的 class属性后：
<li class="item-0" name="link">changedIttem</li>
				<li class="item-0" name="link">changedIttem</li>
				<li class="item-0" name="link">changedIttem</li>
				<li class="item-0" name="link">changedIttem</

<font face="黑体" color=green size=5>2.attr text html</font>

In [40]:
print("文本调整：")
lis2.text("changedIttem")
print(lis2)
print("修改属性：")
lis2.attr('name','link')
print(lis2)

文本调整：
<li class="item-0" name="link">changedIttem</li>
				<li class="item-0" name="link">changedIttem</li>
				<li class="item-0" name="link">changedIttem</li>
				<li class="item-0" name="link">changedIttem</li>
				<li class="item-0" name="link">changedIttem</li>
				<a href="javascript:void(0)" class="item-0 search-reveal" name="link">changedIttem</a> 
			
修改属性：
<li class="item-0" name="link">changedIttem</li>
				<li class="item-0" name="link">changedIttem</li>
				<li class="item-0" name="link">changedIttem</li>
				<li class="item-0" name="link">changedIttem</li>
				<li class="item-0" name="link">changedIttem</li>
				<a href="javascript:void(0)" class="item-0 search-reveal" name="link">changedIttem</a> 
			


In [5]:
#remove()方法实现文本节点的剔除
h = '<div>aaaaaa<p>bbbbbbb</p></div>'
from pyquery import PyQuery
doc = PyQuery(h)
doc.find('p').remove()#剔除不要的p节点
print(doc.text())

aaaaaa


remove()方法实现了剔除不需要的html格式文本，选择自己需要的

另外，其实还有很多节点操作的方法，比如append()、empty()和 prepend()等方法，它们和 jQuery 的用法完全一致

<font face="黑体" color=green size=5> 3.伪类选择器 </font>

In [18]:
print(lis2('li:last-child'))

更多内容,参考pyquery的官方文档：[http://pyquery.readthedocs.io](http://pyquery.readthedocs.io)